In [1]:
import torch
import cv2
import numpy as np
from bipedal_walker import BipedalWalker2
from oneshot import VisualModel
import os

In [25]:
def normalize(img):
    img = img.astype('float32')
    img /= 255.
    return img

trained_model_path = os.path.expanduser("~/Documents/imitation-learning/models/model-naive-all-900.pth")

os.environ["DISPLAY"]=":99"
num_epochs = 1000 
dims = (150, 135)
save_path = "./test_no_state1/"

#model = VisualModel().cuda()
model = torch.load(trained_model_path).cuda()
bipedal = BipedalWalker2(actor_path='./ddpg_bipedal/checkpoint_actor2.pth', critic_path='./ddpg_bipedal/checkpoint_critic2.pth')

results = []
for j in range(20):
    # init
    state = torch.as_tensor(bipedal.env.reset().astype('float32')).unsqueeze(0).cuda()
    env_img = bipedal.env.render(mode='rgb_array')
    # CHANGE POS
    """
    bipedal.env.hull.position[0] = 14
    for i in range(5):
        bipedal.env.step(action)  
    # END CHANGE POS
    """

    env_img = cv2.resize(env_img, dims)
    env_img = normalize(np.transpose(env_img, (2, 0, 1)).copy())
    env_img = torch.as_tensor(env_img)
    env_img = env_img.unsqueeze(0).cuda()

    for i in range(num_epochs):
        action = model(env_img, state)
        action = action.detach().cpu().numpy()[0]
        next_state, done = bipedal.move_bipedal(action)
        next_state = next_state.astype('float32')
        state =  torch.as_tensor(next_state).unsqueeze(0).cuda()
        env_img = normalize(bipedal.save_img(i, save_path))
        env_img = cv2.resize(env_img, dims)
        env_img = np.expand_dims(np.transpose(env_img, (2, 0, 1)), axis=0)
        env_img = torch.as_tensor(env_img).cuda()
        if done:
            print(i)
            results.append(i)
            break

bipedal.close()

print(results)

136
153
122
120
141
140
185
143
121
217
153
100
164
138
147
129
141
83
148
[136, 153, 122, 120, 141, 140, 185, 143, 121, 217, 153, 100, 164, 138, 147, 129, 141, 83, 148]


In [24]:
results_1601 = [164, 90, 165, 150, 149, 106, 129, 131, 126, 158, 94, 150, 110, 107, 141, 140, 126, 116, 137, 115]
sum(results) / len(results)

130.2